In [ ]:
pip install transformers datasets torch accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import json
from datasets import Dataset

# Load JSON dataset
with open("java_bug_fix_dataset.json", "r") as file:
    data = json.load(file)["dataset"]

# Convert to Hugging Face Dataset format
hf_dataset = Dataset.from_list(data)

# Split into train and validation sets
split_dataset = hf_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
valid_dataset = split_dataset["test"]


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load pre-trained model and tokenizer
model_name = "Salesforce/codet5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["buggy_code"], text_target=examples["fixed_code"], padding="max_length", truncation=True)

# Tokenize dataset
tokenized_datasets = hf_dataset.map(tokenize_function, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer

# Define training arguments
training_args = TrainingArguments(
    output_dir="./finetuned_bug_fixer",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2
)

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer
)

# Start training



Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-34-86ae3f588ea5>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
!wandb login


wandb: Currently logged in as: bharathc-23aim (thanushuiav-21cse-kongu-engineering-college) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
trainer.train()

ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided []

In [ ]:
def tokenize_function(examples):
    model_inputs = tokenizer(
        examples["buggy_code"],
        padding="max_length",
        truncation=True,
        max_length=512  # Adjust max length as needed
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["fixed_code"],
            padding="max_length",
            truncation=True,
            max_length=512
        )["input_ids"]

    model_inputs["labels"] = labels
    return model_inputs


In [ ]:
tokenized_datasets = hf_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
print(hf_dataset[0])  # Ensure data is not empty


{'bug_type': 'Logical Error', 'buggy_code': 'public class Main {\n  public static void main(String[] args) {\n    int x = 5;\n    if (x = 10) { // Assignment instead of comparison\n      System.out.println("X is 10");\n    }\n  }\n}', 'fixed_code': 'public class Main {\n  public static void main(String[] args) {\n    int x = 5;\n    if (x == 10) { // Corrected comparison operator\n      System.out.println("X is 10");\n    }\n  }\n}'}


In [ ]:
trainer.train()

ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided []

In [ ]:
print(hf_dataset[0])  # Check if the dataset is loaded correctly
print(tokenized_datasets[0])  # Check if tokenization is working
print(len(train_dataset), len(valid_dataset))  # Check dataset sizes


{'bug_type': 'Logical Error', 'buggy_code': 'public class Main {\n  public static void main(String[] args) {\n    int x = 5;\n    if (x = 10) { // Assignment instead of comparison\n      System.out.println("X is 10");\n    }\n  }\n}', 'fixed_code': 'public class Main {\n  public static void main(String[] args) {\n    int x = 5;\n    if (x == 10) { // Corrected comparison operator\n      System.out.println("X is 10");\n    }\n  }\n}'}
{'bug_type': 'Logical Error', 'buggy_code': 'public class Main {\n  public static void main(String[] args) {\n    int x = 5;\n    if (x = 10) { // Assignment instead of comparison\n      System.out.println("X is 10");\n    }\n  }\n}', 'fixed_code': 'public class Main {\n  public static void main(String[] args) {\n    int x = 5;\n    if (x == 10) { // Corrected comparison operator\n      System.out.println("X is 10");\n    }\n  }\n}', 'input_ids': [1, 482, 667, 12740, 288, 203, 225, 1071, 760, 918, 2774, 12, 780, 8526, 833, 13, 288, 203, 565, 509, 619, 273,

In [ ]:
def tokenize_function(examples):
    model_inputs = tokenizer(
        examples["buggy_code"],
        padding="max_length",
        truncation=True,
        max_length=512  # Adjust if needed
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["fixed_code"],
            padding="max_length",
            truncation=True,
            max_length=512
        )["input_ids"]

    model_inputs["labels"] = labels
    return model_inputs


In [ ]:
tokenized_datasets = hf_dataset.map(tokenize_function, batched=True)
print(tokenized_datasets[0])


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'bug_type': 'Logical Error', 'buggy_code': 'public class Main {\n  public static void main(String[] args) {\n    int x = 5;\n    if (x = 10) { // Assignment instead of comparison\n      System.out.println("X is 10");\n    }\n  }\n}', 'fixed_code': 'public class Main {\n  public static void main(String[] args) {\n    int x = 5;\n    if (x == 10) { // Corrected comparison operator\n      System.out.println("X is 10");\n    }\n  }\n}', 'input_ids': [1, 482, 667, 12740, 288, 203, 225, 1071, 760, 918, 2774, 12, 780, 8526, 833, 13, 288, 203, 565, 509, 619, 273, 1381, 31, 203, 565, 309, 261, 92, 273, 1728, 13, 288, 368, 15499, 3560, 434, 5826, 203, 1377, 2332, 18, 659, 18, 8222, 2932, 60, 353, 1728, 8863, 203, 565, 289, 203, 225, 289, 203, 97, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
from transformers import DataCollatorForSeq2Seq
import torch

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

train_dataset = tokenized_datasets["train"].with_format("torch")
valid_dataset = tokenized_datasets["test"].with_format("torch")


KeyError: "Column train not in the dataset. Current columns in the dataset: ['bug_type', 'buggy_code', 'fixed_code', 'input_ids', 'attention_mask', 'labels']"

In [ ]:
tokenized_datasets = hf_dataset.map(tokenize_function, batched=True)
print(tokenized_datasets[0])  # Should contain `input_ids` and `labels`


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

{'bug_type': 'Logical Error', 'buggy_code': 'public class Main {\n  public static void main(String[] args) {\n    int x = 5;\n    if (x = 10) { // Assignment instead of comparison\n      System.out.println("X is 10");\n    }\n  }\n}', 'fixed_code': 'public class Main {\n  public static void main(String[] args) {\n    int x = 5;\n    if (x == 10) { // Corrected comparison operator\n      System.out.println("X is 10");\n    }\n  }\n}', 'input_ids': [1, 482, 667, 12740, 288, 203, 225, 1071, 760, 918, 2774, 12, 780, 8526, 833, 13, 288, 203, 565, 509, 619, 273, 1381, 31, 203, 565, 309, 261, 92, 273, 1728, 13, 288, 368, 15499, 3560, 434, 5826, 203, 1377, 2332, 18, 659, 18, 8222, 2932, 60, 353, 1728, 8863, 203, 565, 289, 203, 225, 289, 203, 97, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
from datasets import DatasetDict

# Define the split ratio (e.g., 80% training, 20% testing)
split_ratio = 0.8
split_index = int(len(tokenized_datasets) * split_ratio)

train_dataset = tokenized_datasets.select(range(split_index))
valid_dataset = tokenized_datasets.select(range(split_index, len(tokenized_datasets)))

# Convert to DatasetDict
dataset = DatasetDict({
    "train": train_dataset,
    "test": valid_dataset
})


In [ ]:
train_dataset = dataset["train"].with_format("torch")
valid_dataset = dataset["test"].with_format("torch")


In [ ]:
print("Train dataset example:", train_dataset[0])
print("Valid dataset example:", valid_dataset[0])
print("Train dataset size:", len(train_dataset))
print("Valid dataset size:", len(valid_dataset))


Train dataset example: {'bug_type': 'Logical Error', 'buggy_code': 'public class Main {\n  public static void main(String[] args) {\n    int x = 5;\n    if (x = 10) { // Assignment instead of comparison\n      System.out.println("X is 10");\n    }\n  }\n}', 'fixed_code': 'public class Main {\n  public static void main(String[] args) {\n    int x = 5;\n    if (x == 10) { // Corrected comparison operator\n      System.out.println("X is 10");\n    }\n  }\n}', 'input_ids': tensor([    1,   482,   667, 12740,   288,   203,   225,  1071,   760,   918,
         2774,    12,   780,  8526,   833,    13,   288,   203,   565,   509,
          619,   273,  1381,    31,   203,   565,   309,   261,    92,   273,
         1728,    13,   288,   368, 15499,  3560,   434,  5826,   203,  1377,
         2332,    18,   659,    18,  8222,  2932,    60,   353,  1728,  8863,
          203,   565,   289,   203,   225,   289,   203,    97,     2,     0,
            0,     0,     0,     0,     0,     0,     0,  

In [ ]:
def tokenize_function(examples):
    inputs = [ex["buggy_code"] for ex in examples]  # Input: buggy code
    targets = [ex["fixed_code"] for ex in examples]  # Target: fixed code

    # Tokenize inputs and targets
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True)
    labels = tokenizer(targets, padding="max_length", truncation=True)

    # Add labels
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
def tokenize_function(examples):
    inputs = examples["buggy_code"]  # List of buggy code samples
    targets = examples["fixed_code"]  # List of corresponding fixed code samples

    # Tokenize inputs and targets
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True)
    labels = tokenizer(targets, padding="max_length", truncation=True)

    # Add labels
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_datasets["train"][0])  # Should include `input_ids`, `attention_mask`, and `labels`


{'bug_type': 'Logical Error', 'buggy_code': 'public class Main {\n  public static void main(String[] args) {\n    int x = 5;\n    if (x = 10) { // Assignment instead of comparison\n      System.out.println("X is 10");\n    }\n  }\n}', 'fixed_code': 'public class Main {\n  public static void main(String[] args) {\n    int x = 5;\n    if (x == 10) { // Corrected comparison operator\n      System.out.println("X is 10");\n    }\n  }\n}', 'input_ids': [1, 482, 667, 12740, 288, 203, 225, 1071, 760, 918, 2774, 12, 780, 8526, 833, 13, 288, 203, 565, 509, 619, 273, 1381, 31, 203, 565, 309, 261, 92, 273, 1728, 13, 288, 368, 15499, 3560, 434, 5826, 203, 1377, 2332, 18, 659, 18, 8222, 2932, 60, 353, 1728, 8863, 203, 565, 289, 203, 225, 289, 203, 97, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
from transformers import TrainingArguments, Trainer

# Define training arguments
training_args = TrainingArguments(
    output_dir="./finetuned_bug_fixer",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2
)

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer
)

# Start training
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-63-737cdbb93efd>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
